<a href="https://colab.research.google.com/github/1920643/1920643.github.io/blob/main/NikeVsAdidas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Dénes Csala**  
University of Bristol, 2021  

Based on *Elements of Data Science* ([Allen B. Downey](https://allendowney.com), 2021) and *Python Data Science Handbook* ([Jake VanderPlas](https://jakevdp.github.io/PythonDataScienceHandbook/), 2018)

License: [MIT](https://mit-license.org/)

# Loading financial data into _pandas_

Install _Yahoo Finance_. If you put a `!` in the first character of a cell, it becomes a _Linux_ command. The cell below would install the `yfinance` _python package_, using the `pip` package manager tool. However, when you run the cell for the second time (in _Colab_ the packages you install persist for about 8 hours) it is already installed - so you get only `Requirement already satisfied` messages.

In [1]:
!pip install yfinance

     |████████████████████████████████| 6.3 MB 6.4 MB/s 
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [2]:
import yfinance as yf

In [3]:
Nike = yf.Ticker("NKE").history(period='5y')
Adidas = yf.Ticker("ADDYY").history(period='5y')


Now we have downloaded the data for four stocks, computer graphics company nVidia `NVDA` and electric vehicle manufacturer Tesla `TSLA`, and low-cost airlines RyanAir `RYAAY` and WizzAir `WIZZ.L` for the past 5 years.

The responses returned are _pandas_ `DataFrames`. They contain [OHLC](https://www.investopedia.com/terms/o/ohlcchart.asp) data, but we only need the `Close` columns this time. Let us also give them names.

In [4]:
Nike

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2016-12-09,49.137049,49.336410,48.890227,49.099079,5869800,0.000,0
2016-12-12,48.890222,49.269953,48.358599,48.928196,11459500,0.000,0
2016-12-13,49.203497,50.523058,49.203497,49.649677,11442300,0.000,0
2016-12-14,49.592724,49.687655,48.975662,49.165527,8012800,0.000,0
2016-12-15,49.364888,49.402862,48.501003,48.690868,9567500,0.000,0
...,...,...,...,...,...,...,...
2021-12-02,166.979886,170.244024,166.700390,169.695007,4398900,0.000,0
2021-12-03,170.619995,172.070007,166.759995,170.240005,4878200,0.305,0
2021-12-06,170.830002,171.210007,168.059998,168.910004,5636900,0.000,0


In [5]:
Nike=Nike[['Close']]
Nike['Name']='Nike'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [6]:
Nike.head()

,Close,Name
Date,,
2016-12-09,49.099079,Nike
2016-12-12,48.928196,Nike
2016-12-13,49.649677,Nike
2016-12-14,49.165527,Nike
2016-12-15,48.690868,Nike


This is what we want. However surely, there is a more efficient way to automating this, if we have multiple stocks.

In [7]:
dfs=[] #create empty list of dataframes
for x in ['NKE','ADDYY']:
  df = yf.Ticker(x).history(period='5y')
  df=df[['Close']]
  df['Name']=x
  dfs.append(df) #append newly download and formatted dataframe to our list of dataframes

Great. Now we have a list of `DataFrame`s, each containing the closing stock price and the stock name, for the past 5 years. 

# DataFrame combination

## Concatenation

In [8]:
dfs[0].head(2)

,Close,Name
Date,,
2016-12-09,49.099087,NKE
2016-12-12,48.928200,NKE


In [9]:
dfs[1].head(2)

,Close,Name
Date,,
2016-12-09,72.536430,ADDYY
2016-12-12,72.898254,ADDYY


We can combine `DataFrames` by stacking them on top of each other using `concat`. They must have the same `column` names (otherwise, empty columns will be created and filled with `NaN`s). The `pd.concat` function accepts only `list []` arguments - therefore, the `DataFrame`s to be combined have to be in the format `[dfA, dfB]`.

In [10]:
import pandas as pd
pd.concat([ dfs[0],dfs[1] ])

,Close,Name
Date,,
2016-12-09,49.099087,NKE
2016-12-12,48.928200,NKE
2016-12-13,49.649689,NKE
2016-12-14,49.165539,NKE
2016-12-15,48.690865,NKE
...,...,...
2021-12-02,145.179993,ADDYY
2021-12-03,141.720001,ADDYY
2021-12-06,144.059998,ADDYY


## Joining

Sometimes, the dataframes to be joined need to end up next to each other, a "_horizontal_ `concat`". This is called `join`. The `DataFrames` to be combined must have the same index. They must _not_ have any matching `column` names - though these can be renamed automatically using `lsuffix` or `rsuffix`.

Let's do something smarter:

In [22]:
dfy=pd.DataFrame() #initialise empty DataFrame
for x in dfs:
  stock_name=x['Name'].values[0]
  stock_name=stock_name.replace('.','')
  x=x[['Close']]
  x.columns=[stock_name]
  dfy=x.join(dfy)

In [20]:
dfz.head()

,ADDYY,NKE
Date,,
2016-12-09,72.536430,49.099087
2016-12-12,72.898254,48.928200
2016-12-13,73.717117,49.649689
2016-12-14,71.460480,49.165539
2016-12-15,71.279572,48.690865


Ready to export. In _CSV_:

In [23]:
dfy.to_csv('stocks.csv')

To _JSON_.

In [24]:
import json

In [25]:
json_list_of_dicts=list(df.T.to_dict().values())
open('stocks.json','w').write(json.dumps(json_list_of_dicts))

59541